# Heat Equation

The heat equation is a typical example of parabolic equations. We search for the temperature $u : \Omega \times[0,T] \rightarrow R$ satisfying 

$$
\frac{\partial u}{\partial t}(x,t) - \Delta u(x,t) = f(x,t) \qquad \forall \, x \in \Omega, \forall t \in (0,T),
$$

where $f$ is a given heat source density. Here, $\Omega \subset R^d$ is the spatial domain, and $[0,T]$ is the time interval. In addition, we need initial conditions 

$$
u(x,0) = u_0(x) \qquad \forall \, x \in \Omega,
$$

and boundary conditions at the spatial boundary. For example Dirichlet boundary conditions read as

$$
u(x,t) = u_D(x,t) \qquad \forall \, x \in \partial \Omega, \; \forall  t \in(0,T]
$$

## Variational formulation in space

Similar as for elliptic equations we pass over to the weak formulations. We do this only for the spatial variables $x$, for all $t \in (0,T)$. The test functions $v$ depend only on $x$. We multiply by $v$, integration over $\Omega$, and perform integration by parts to obtain:

$$
\int_\Omega \frac{\partial u}{\partial t}(x,t) v(x) \, dx + \int \nabla u(x,t) \nabla v(x) \, dx = \int_\Omega f(x,t) v(x) \, dx \qquad \forall \, v \in H_0^1(\Omega) \; \forall t \, \in (0,T)
$$

The initial condition in $L_2$ is set for $t = 0$:
$$
u(\cdot, 0) = u_0 \qquad \text{in} \; L_2(\Omega)
$$

We consider the unknown function $u$ as a function with values in the Hilbert space $H^1$, e.g. $u : [0,T] \rightarrow H^1(\Omega)$. This allows to express the equation as Hilbert-space valued ordinary differential equation:
$$
\frac{d u }{dt }(t) + A u(t) = f(t)
$$
... some more definitions to come ...

## Galerkin method in space

As a next step we apply the finite element method in space. Let $V_h$ be a finite element sub-space of $H^1(\Omega)$, with the basis functions $\{ p_1(x), \ldots p_N(x) \}$. We approximate the time-dependent function by

$$
u_h : [0,T] \rightarrow V_h,
$$

which can we expressed by means of the basis as

$$
u_h(t) = \sum_{i=1}^N u_i(t) p_i(x)
$$

Inserting this expansion into the variational formulation, and using spatial test-functions $v = p_j$ we obtain the semi-discrete equation:


$$
\int_\Omega \frac{\partial}{\partial t} \Big\{ \sum_{i=1}^N u_i(t) p_i(x) \Big\} p_j(x) dx + 
\int_\Omega \nabla \Big\{ \sum_{i=1}^N u_i(t) p_i(x) \Big\} \; \nabla p_j(x) = \int_\Omega f(x,t) p_j(x) \, dx
\qquad \forall \, j = 1 \ldots N, \forall \, t \in (0,T)
$$


The initial condition for $u_h$ is obtained by some kind of projection, typically we want the $L_2$ projection

$$
\int u_h(x,0) p_j(x) \, dx = \int u_0(x) p_j(x) \, dx \qquad \forall \, j \in \{ 1, \ldots N \}
$$





By means of the **mass matrix** $M \in R^{N \times N}$ 
$$
M = \left\{ \int_\Omega p_i p_j \, dx \right\}_{i,j = 1, \ldots N}
$$
and **stiffness matrix** $A \in R^{N \times N}$ 
$$
A = \left\{ \int_\Omega \nabla p_i \nabla p_j \, dx \right\}_{i,j = 1, \ldots N}
$$
we obtain the ordinary differential equation (ODE)

$$
M \dot u(t) + A u(t) = f(t) \qquad \forall \, t 
$$

including the initial condition 

$$
u(0) = u_0
$$

for the unknown coefficient function $u(\cdot) = (u_1(\cdot), \ldots u_N(\cdot)) \in C^1([0,T], R^N)$.

# Implicit Euler time-stepping

We approximate the function $u$ at time-steps $t_j = j \tau$ by $u^j$, with the time-step $\tau$ and $j \in \{0, \ldots m\}$. By replacing the time-derivative $\dot u$ by the backward difference quotient we obtain

$$
M \frac{u^{j}-u^{j-1}}{\tau} + A u^j = f^j
$$

The initial value $u^0 := u_0$ is given by the initial condition. If the old time-step solution $u^{j-1}$ is known, we can compute the next step by the linear system of equations

$$
(M + \tau A) u^j = \tau f + M u^{j-1}.
$$

Often the new step is computed in incremental form:

\begin{eqnarray*}
(M + \tau A) w^j &=& f - A u^{j-1} \\
u^j & = & u^{j-1} + \tau w^j
\end{eqnarray*}

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
from time import sleep

setting up matrices:

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.01))
tau = 0.005
tend = 0.1
u0 = exp(-100*( (x-0.5)**2 + (y-0.5)**2))

fes = H1(mesh, order=5)
u,v = fes.TnT()
mform = u*v*dx
aform = grad(u)*grad(v)*dx

ecact_m = BilinearForm(mform).Assemble()
exact_a = BilinearForm(aform).Assemble()
exact_mstar = BilinearForm(mform+tau*aform).Assemble()
exact_mstarinv = exact_mstar.mat.Inverse()
exact_f = LinearForm(fes).Assemble()

exact_gfu = GridFunction(fes)
exact_gfu.Set(u0)

scene = Draw (exact_gfu, deformation=True)
sleep(3)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [8]:
tau = 0.005
tend = 0.1
u0 = exp(-100*( (x-0.5)**2 + (y-0.5)**2))

mesh = Mesh(unit_square.GenerateMesh(maxh=0.0625))
fes = H1(mesh, order=2)
u,v = fes.TnT()
mform = u*v*dx
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform).Assemble()
a = BilinearForm(aform).Assemble()
mstar = BilinearForm(mform+tau*aform).Assemble()
mstarinv = mstar.mat.Inverse()
f = LinearForm(fes).Assemble()

gfu = GridFunction(fes)
gfu.Set(u0)

scene = Draw (gfu, deformation=True)
sleep(3)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [9]:
errors = []
for j in range(int(tend/tau)):
    exact_res = exact_f.vec - exact_a.mat * exact_gfu.vec
    exact_w = exact_mstarinv * exact_res
    exact_gfu.vec.data += tau*exact_w
    res = f.vec - a.mat * gfu.vec
    w = mstarinv * res
    gfu.vec.data += tau*w
    err = sqrt (Integrate ( (gfu-exact_gfu)*(gfu-exact_gfu), mesh))
    errors.append(err)
#     scene.Redraw()
#     sleep(0.2)

In [10]:
errors

[0.07322928967556033,
 0.051908263105424395,
 0.03962798256728202,
 0.03135435372242473,
 0.02528169602783462,
 0.02061054236025338,
 0.016919765976712464,
 0.013955340372788424,
 0.01154852502312929,
 0.009579764780180825,
 0.007960636854257254,
 0.006623771821004474,
 0.005516718594109344,
 0.004597966766068795,
 0.0038342462706870543,
 0.00319862819041557,
 0.0026691495895909984,
 0.0022277925160244213,
 0.00185970916600863,
 0.001552622585288854]

In [6]:
mesh_size = [0.5, 0.25, 0.125, 0.0625]#, 0.03125, 0.015625]
dofs = []
errors = []

for h in mesh_size:
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))
    fes = H1(mesh, order=2)
    u,v = fes.TnT()
    mform = u*v*dx
    aform = grad(u)*grad(v)*dx

    m = BilinearForm(mform).Assemble()
    a = BilinearForm(aform).Assemble()
    mstar = BilinearForm(mform+tau*aform).Assemble()
    mstarinv = mstar.mat.Inverse()
    f = LinearForm(fes).Assemble()

    gfu = GridFunction(fes)
    gfu.Set(u0)
    dofs.append(fes.ndof)
    errs = []
    for j in range(int(tend/tau)):
        exact_res = exact_f.vec - exact_a.mat * exact_gfu.vec
        exact_w = exact_mstarinv * exact_res
        exact_gfu.vec.data += tau*exact_w
        res = f.vec - a.mat * gfu.vec
        w = mstarinv * res
        gfu.vec.data += tau*w
        err = sqrt (Integrate ( (gfu-exact_gfu)*(gfu-exact_gfu), mesh))
        errs.append(err)
    errors.append(errs)

In [7]:
errors

[[0.24866694975016188,
  0.19849668528103137,
  0.16499422379753664,
  0.14168206977787687,
  0.1254119767827915,
  0.11419351927434415,
  0.10658695322635053,
  0.10151315169792184,
  0.09817474004013528,
  0.0960007799527952,
  0.09459540854665709,
  0.0936913246472018,
  0.09311153390249212,
  0.09274041720622624,
  0.09250312490545849,
  0.0923514804674134,
  0.0922545872622218,
  0.09219267378399609,
  0.09215310352694893,
  0.0921278053507782],
 [0.07273550804702254,
  0.05182877484103195,
  0.03973749487288559,
  0.031544453749076766,
  0.025510201006176184,
  0.02086713637624686,
  0.017206985934569412,
  0.014279504978841251,
  0.011916228493300353,
  0.009996876732166562,
  0.008432197448834285,
  0.007154093414481829,
  0.006109451543772491,
  0.00525606937551372,
  0.0045598659321218205,
  0.003992926240329312,
  0.003532114684249132,
  0.003158092688422954,
  0.002854628687397931,
  0.002608113765895322],
 [0.07363323554210924,
  0.052006393422671326,
  0.03965830783879989